In [1]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-finalexam-p3/train.csv
/kaggle/input/2022-ml-finalexam-p3/test.csv
/kaggle/input/2022-ml-finalexam-p3/sample_submit.csv


In [2]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [3]:
submit = pd.read_csv('../input/2022-ml-finalexam-p3/sample_submit.csv')
test = pd.read_csv('../input/2022-ml-finalexam-p3/test.csv')
train = pd.read_csv('../input/2022-ml-finalexam-p3/train.csv')

In [4]:
# 식단 관련된 열 따로 저장
train_morning = pd.DataFrame(train['조식메뉴'])
train_lunch = pd.DataFrame(train['중식메뉴'])
train_dinner = pd.DataFrame(train['석식메뉴'])

test_morning = pd.DataFrame(test['조식메뉴'])
test_lunch = pd.DataFrame(test['중식메뉴'])
test_dinner = pd.DataFrame(test['석식메뉴'])

In [5]:
# 요일 실수화
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['요일'] = le.fit_transform(train['요일'])
test['요일'] = le.transform(test['요일'])

In [6]:
# 쓰지 않는 칼럼 drop. 조식메뉴, 중식메뉴, 석식메뉴는 추후에 변환 후 다시 concat
trainX = train.drop(['Unnamed: 0', '일자', '조식메뉴', '중식메뉴', '석식메뉴', '중식계'], axis = 1)
trainY = train['중식계']
testX = test.drop(['Unnamed: 0', '일자', '조식메뉴', '중식메뉴', '석식메뉴'], axis = 1)

In [7]:
# train 식단 전부 합치기
train_menu = pd.concat([train_morning, train_lunch, train_dinner], axis = 1)
# test 식단 전부 합치기
test_menu = pd.concat([test_morning, test_lunch, test_dinner], axis = 1)
# train, test 식단 전부 합치기
menu = pd.concat([train_menu, test_menu], axis = 0)

In [8]:
# countvectorizer를 통하여 text data 처리

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 100)
# 전체 식단으로 fit
vectorizer.fit(menu)

# train 각각의 식단을 넘파이배열로 변환 후 transform
train_morning = vectorizer.transform(np.array(train_morning).flatten())
train_lunch = vectorizer.transform(np.array(train_lunch).flatten())
train_dinner = vectorizer.transform(np.array(train_dinner).flatten())

# 벡터라이저 적용 후 다시 dataframe으로 변환
train_morning = pd.DataFrame(train_morning.toarray())
train_lunch = pd.DataFrame(train_lunch.toarray())
train_dinner = pd.DataFrame(train_dinner.toarray())

# test 각각의 식단을 넘파이배열로 변환 후 transform
test_morning = vectorizer.transform(np.array(test_morning).flatten())
test_lunch = vectorizer.transform(np.array(test_lunch).flatten())
test_dinner = vectorizer.transform(np.array(test_dinner).flatten())

# 벡터라이저 적용 후 다시 dataframe으로 변환
test_morning = pd.DataFrame(test_morning.toarray())
test_lunch = pd.DataFrame(test_lunch.toarray())
test_dinner = pd.DataFrame(test_dinner.toarray())

In [9]:
# 벡터라이저 변환이 완료된 식단들을 원래 데이터들과 합체
trainX = pd.concat([trainX, train_morning, train_lunch, train_dinner], axis = 1)
testX = pd.concat([testX, test_morning, test_lunch, test_dinner], axis = 1)

In [10]:
# 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [11]:
trainX.shape[0]

843

In [12]:
# randomfrorestregressor 사용
from sklearn.ensemble import RandomForestRegressor
# 인스턴스 생성
rfr = RandomForestRegressor(random_state = seed, max_depth = 10)
# train data로 학습
rfr.fit(trainX, trainY)
# test data를 예측
pred = rfr.predict(testX)

In [13]:
submit['중식계'] = pred
submit.to_csv('submission.csv', index = False)